In [9]:
import langchain
print(langchain.__version__)

1.2.10


# Models

In [10]:
from langchain.chat_models import init_chat_model

model = init_chat_model(
    model="qwen2.5-coder:7b",
    model_provider="ollama",
    temperature = 0.0
)
print(model.profile)

None


In [ ]:
# model = init_chat_model(
#     model='gpt-oss:20b',
#     model_provider="ollama",
#     temperature=0
# )

In [12]:

response = model.invoke("Why do parrots talk?")
print(response)

content="Parrots are known for their ability to mimic human speech and sounds due to several biological and behavioral factors:\n\n1. **Brain Structure**: Parrots have a highly developed brain structure called the nidopallium, which is responsible for learning and vocalization. This area of the brain allows them to imitate complex sounds.\n\n2. **Repetition Learning**: Like many birds, parrots learn through repetition. They hear human voices and other sounds repeatedly in their environment and try to mimic these sounds as a way of communication or social bonding.\n\n3. **Social Behavior**: Parrots are highly social animals and often engage in vocalizations as part of their communication with others. Mimicking human speech can be a form of play, learning, or even expressing emotions.\n\n4. **Curiosity**: Parrots are naturally curious about their environment and the sounds they hear. They may mimic human speech out of curiosity rather than any specific need to communicate.\n\n5. **Cultur

## Prompts

In [13]:
response = model.invoke("Move Cube To Destination zone 1, from the last sentence only give the verbs")
print(response)

content='move, to, destination, zone, one' additional_kwargs={} response_metadata={'model': 'qwen2.5-coder:7b', 'created_at': '2026-02-18T15:15:23.0655815Z', 'done': True, 'done_reason': 'stop', 'total_duration': 2619671300, 'load_duration': 101696100, 'prompt_eval_count': 45, 'prompt_eval_duration': 1436975900, 'eval_count': 10, 'eval_duration': 1029336500, 'logprobs': None, 'model_name': 'qwen2.5-coder:7b', 'model_provider': 'ollama'} id='lc_run--019c7151-ecfc-72c3-a259-a0cd3b0745aa-0' tool_calls=[] invalid_tool_calls=[] usage_metadata={'input_tokens': 45, 'output_tokens': 10, 'total_tokens': 55}


In [14]:
from langchain_core.prompts import ChatPromptTemplate

system_prompt = """
You are a extractor who will extract only the object and destination zone from a user prompt, 

your main tasks are as follows:
1. Remove all specials characters and give space between destination zone and the corresponding number
2. From the user prompt extract the object that need to be moved or placed, and check whether it is in ["cube", "brief case", "first aid kit", "paracetamol", "sugar"], if the extracted
   word is not in your list reply "object ['object name'] not found please enter another object".
3. The user may join the names togther without space give more space, or there will be under score between object name, handle all that perfectly
4. All the outputs should be in lower case.
5. If The user enter a object that is in present in this [{moved_objects}] reply that "Object already moved Cannot move again"
"""

chat_history = [
    ('system',system_prompt),
    ('human','move oil to drop_zone1'),
    ('ai','oil, drop zone 1'),
    ('human','move BRIEF_CASE to dropzone1'),
    ('ai','brief case, drop zone 1'),
    ('human','move ball to drop zone1'),
    ('ai','Object [ball] cannot be moved to drop zone 1 as its not in the current environment'),
    ('human','move Sugar to drop zone_2'),
    ('ai','sugar, drop zone 2'),
    ('human','move brazil to drop zone3'),
    ('ai','Object [brazil] cannot be moved to drop zone 3 as its not in the current environment'),
    ('human','move First aidkit to dropzone-3'),
    ('ai','first aid kit, drop zone 3'),
    ('human','{user_input}')
]

moved_objects = ['oil','sugar']


while True :
    template = ChatPromptTemplate(messages=chat_history)
    
    user_input = input("Command")
    if user_input == "exit":
        break
    

    prompt = template.invoke({
        "user_input" : user_input,
        "moved_objects":moved_objects
    })

    chat_history.insert(-1, ('human',user_input))
    response = model.invoke(prompt)

    chat_history.insert(-1, ('ai',response.content))
    print(len(str(response.content).split(' ')))
    if len(str(response.content).split(' ')) <= 4 :
        moved_objects.append(str(response.content).split(',')[0])
    
    print('Moved Objects :',moved_objects)
    for key in response:
        print(key)
    print("Chat History :",chat_history)


## Messages

In [15]:
from langchain_core.messages import SystemMessage, HumanMessage, AIMessage

## Structured Output

In [16]:
from typing import Literal
from pydantic import BaseModel, Field

from langchain_core.prompts import ChatPromptTemplate

system_prompt = """
You are a extractor who will extract only the object and destination zone from a user prompt, 

your main tasks are as follows:
1. Remove all specials characters, unwanted spaces and lower case the user prompt.
2. From the user prompt extract the object that need to be moved or placed, and check whether it is in {available_objects}, if not 
    "object" = "Not Valid", "location" = "extracted_location", "status" = "Object 'extracted object' not available, please select another object from {available_objects}".
3. From the user prompt extract the location where the object need to be moved or placed, and check whether it is in {drop_locations}, if not 
    "object" = "extracted_object", "location" = "Not Valid", "status" = "Location not available, please select another object from {drop_locations}.
4. If The user enter a object that is in present in this [{moved_objects}] then
    "object" = "Not Valid", "location" = "extracted_location", "status" = "Object 'extracted object' has already moved cannot move again, please select another object from {available_objects}"
5. If the both extracted object and location not in {available_objects} and {drop_locations} respectively, then
    "object" = "Not Valid", "location" = "Not Valid", "status" = "Both object and location is not valid, please select another object from {available_objects} and location from {drop_locations}"
6. The user may join the names togther without space give more space, or there will be under score between object name, handle all that perfectly


"""

chat_history = [
    ('system',system_prompt),
    
    ('human','move cube to drop_zone1'),
    ('ai','{{ "object":"cube", "location":"drop zone 1", "status":"Valid" }}'),
    
    ('human','move BRIEF_CASE to dropzone1'),
    ('ai','{{ "object":"brief case", "location":"drop zone 1", "status":"Valid"  }}'),
    
    ('human','move ball to drop zone1'),
    ('ai','{{ "object":"Not Valid", "location":"drop zone 1", "status":" "Object ball not available, please select another object from {available_objects}" }}'),
    
    ('human','move pen to drop zone_2'),
    ('ai','{{ "object":"pen", "location":"drop zone 2", "status":"Valid"  }}'),
    
    ('human','move oil to drop zone3'),
    ('ai','{{"object" = "Not Valid", "location" = "drop zone 3", "status" = "Object oil has already moved cannot move again, please select another object from {available_objects}" }}'),
    
    ('human','move First aidkit to dropzone-5'),
    ('ai','{{"object" = "first aid kit", "location" = "Not Valid", "status" = "Location not available, please select another object from {drop_locations}" }}'),
    
    ('human','move ball to dropzone-5'),
    ('ai','{{"object" = "Not Valid", "location" = "Not Valid", "status" = "Both object and location is not valid, please select another object from {available_objects} and location from {drop_locations}" }}'),
    
    ('human','{user_input}')
]

available_objects = ["cube", "brief case", "first aid kit", "paracetamol", "sugar"]
drop_locations = ['drop zone 1', 'drop zone 2', 'drop zone 3', 'drop zone 4']

moved_objects = ['oil','sugar']

template = ChatPromptTemplate(messages=chat_history)

user_input = "move oil to drop zone2"

prompt = template.invoke(
    {
        "moved_objects" : moved_objects,
        "available_objects" : available_objects,
        "drop_locations":drop_locations,
        "user_input" : user_input
    }
)

available_objects_schema = Literal["cube", "brief case", "first aid kit", "paracetamol", "sugar", "Not Valid"]
drop_locations_schema = Literal['drop zone 1', 'drop zone 2', 'drop zone 3', 'drop zone 4', 'Not Valid']

class ExtractorSchema(BaseModel):
    object : available_objects_schema = Field(description="Extracted object from the query")
    location : drop_locations_schema = Field(description="Extracted location from the query")
    status : str = Field(description="Overall status of the operation")
    

struct_model = model.with_structured_output(ExtractorSchema)

response = struct_model.invoke(prompt)
    
    

print(response)
print(type(response))


object='Not Valid' location='drop zone 2' status="Object oil has already moved cannot move again, please select another object from ['cube', 'brief case', 'first aid kit', 'paracetamol', 'sugar']"
<class '__main__.ExtractorSchema'>


## Output Parsers

In [17]:
from langchain_core.output_parsers import JsonOutputParser

from langchain_core.prompts import ChatPromptTemplate

system_prompt = """
You are a extractor who will extract only the object and destination zone from a user prompt, 

your main tasks are as follows:
1. Remove all specials characters, unwanted spaces and lower case the user prompt.
2. From the user prompt extract the object that need to be moved or placed, and check whether it is in {available_objects}, if not 
    "object" : "Not Valid", "location" : "extracted_location", "status" : "Object 'extracted object' not available, please select another object from {available_objects}".
3. From the user prompt extract the location where the object need to be moved or placed, and check whether it is in {drop_locations}, if not 
    "object" : "extracted_object", "location" : "Not Valid", "status" : "Location not available, please select another object from {drop_locations}.
4. If The user enter a object that is in present in this [{moved_objects}] then
    "object" : "Not Valid", "location" : "extracted_location", "status" : "Object 'extracted object' has already moved cannot move again, please select another object from {available_objects}"
5. If the both extracted object and location not in {available_objects} and {drop_locations} respectively, then
    "object" : "Not Valid", "location" : "Not Valid", "status" : "Both object and location is not valid, please select another object from {available_objects} and location from {drop_locations}"
6. The user may join the names togther without space give more space, or there will be under score between object name, handle all that perfectly

{format_instruction}

"""

chat_history = [
    ('system',system_prompt),
    
    ('human','move cube to drop_zone1'),
    ('ai','{{ "object":"cube", "location":"drop zone 1", "status":"Valid" }}'),
    
    ('human','move BRIEF_CASE to dropzone1'),
    ('ai','{{ "object":"brief case", "location":"drop zone 1", "status":"Valid"  }}'),
    
    ('human','move ball to drop zone1'),
    ('ai','{{ "object":"Not Valid", "location":"drop zone 1", "status":" "Object ball not available, please select another object from {available_objects}" }}'),
    
    ('human','move pen to drop zone_2'),
    ('ai','{{ "object":"pen", "location":"drop zone 2", "status":"Valid"  }}'),
    
    ('human','move oil to drop zone3'),
    ('ai','{{"object" : "Not Valid", "location" : "drop zone 3", "status" : "Object oil has already moved cannot move again, please select another object from {available_objects}" }}'),
    
    ('human','move First aidkit to dropzone-5'),
    ('ai','{{"object" : "first aid kit", "location" : "Not Valid", "status" = "Location not available, please select another object from {drop_locations}" }}'),
    
    ('human','move ball to dropzone-5'),
    ('ai','{{"object" : "Not Valid", "location" : "Not Valid", "status" : "Both object and location is not valid, please select another object from {available_objects} and location from {drop_locations}" }}'),
    
    ('human','{user_input}')
]

available_objects = ["cube", "brief case", "first aid kit", "paracetamol", "sugar"]
drop_locations = ['drop zone 1', 'drop zone 2', 'drop zone 3', 'drop zone 4']

moved_objects = ['oil','sugar']
parser = JsonOutputParser()

template = ChatPromptTemplate(messages=chat_history, partial_variables={"format_instruction":parser.get_format_instructions()})

user_input = "move cube to drop zone-6"


prompt = template.invoke(
    {
        "moved_objects" : moved_objects,
        "available_objects" : available_objects,
        "drop_locations":drop_locations,
        "user_input" : user_input
    },
)

response = model.invoke(prompt)
final_response = parser.parse(response.content)
    

# print(response)
# print(type(response.content))
print(final_response)



{'object': 'cube', 'location': 'Not Valid', 'status': "Location not available, please select another object from ['drop zone 1', 'drop zone 2', 'drop zone 3', 'drop zone 4']"}


## Chains

In [18]:
from typing import Literal
from pydantic import BaseModel, Field

from langchain_core.prompts import ChatPromptTemplate

system_prompt = """
You are a extractor who will extract only the object and destination zone from a user prompt, 

your main tasks are as follows:
1. Remove all specials characters, unwanted spaces and lower case the user prompt.
2. From the user prompt extract the object that need to be moved or placed, and check whether it is in {available_objects}, if not 
    "object" = "Not Valid", "location" = "extracted_location", "status" = "Object 'extracted object' not available, please select another object from {available_objects}".
3. From the user prompt extract the location where the object need to be moved or placed, and check whether it is in {drop_locations}, if not 
    "object" = "extracted_object", "location" = "Not Valid", "status" = "Location not available, please select another object from {drop_locations}.
4. If The user enter a object that is in present in this [{moved_objects}] then
    "object" = "Not Valid", "location" = "extracted_location", "status" = "Object 'extracted object' has already moved cannot move again, please select another object from {available_objects}"
5. If the both extracted object and location not in {available_objects} and {drop_locations} respectively, then
    "object" = "Not Valid", "location" = "Not Valid", "status" = "Both object and location is not valid, please select another object from {available_objects} and location from {drop_locations}"
6. The user may join the names togther without space give more space, or there will be under score between object name, handle all that perfectly


"""

chat_history = [
    ('system',system_prompt),
    
    ('human','move cube to drop_zone1'),
    ('ai','{{ "object":"cube", "location":"drop zone 1", "status":"Valid" }}'),
    
    ('human','move BRIEF_CASE to dropzone1'),
    ('ai','{{ "object":"brief case", "location":"drop zone 1", "status":"Valid"  }}'),
    
    ('human','move ball to drop zone1'),
    ('ai','{{ "object":"Not Valid", "location":"drop zone 1", "status":" "Object ball not available, please select another object from {available_objects}" }}'),
    
    ('human','move pen to drop zone_2'),
    ('ai','{{ "object":"pen", "location":"drop zone 2", "status":"Valid"  }}'),
    
    ('human','move oil to drop zone3'),
    ('ai','{{"object" = "Not Valid", "location" = "drop zone 3", "status" = "Object oil has already moved cannot move again, please select another object from {available_objects}" }}'),
    
    ('human','move First aidkit to dropzone-5'),
    ('ai','{{"object" = "first aid kit", "location" = "Not Valid", "status" = "Location not available, please select another object from {drop_locations}" }}'),
    
    ('human','move ball to dropzone-5'),
    ('ai','{{"object" = "Not Valid", "location" = "Not Valid", "status" = "Both object and location is not valid, please select another object from {available_objects} and location from {drop_locations}" }}'),
    
    ('human','{user_input}')
]

available_objects = ["cube", "brief case", "first aid kit", "paracetamol", "sugar"]
drop_locations = ['drop zone 1', 'drop zone 2', 'drop zone 3', 'drop zone 4']

moved_objects = ['oil','sugar']

template = ChatPromptTemplate(messages=chat_history)

user_input = "move oil to drop zone2"


available_objects_schema = Literal["cube", "brief case", "first aid kit", "paracetamol", "sugar", "Not Valid"]
drop_locations_schema = Literal['drop zone 1', 'drop zone 2', 'drop zone 3', 'drop zone 4', 'Not Valid']

class ExtractorSchema(BaseModel):
    object : available_objects_schema = Field(description="Extracted object from the query")
    location : drop_locations_schema = Field(description="Extracted location from the query")
    status : str = Field(description="Overall status of the operation")
    

struct_model = model.with_structured_output(ExtractorSchema)

chain = template | struct_model 

chain.invoke(
    {
        "moved_objects" : moved_objects,
        "available_objects" : available_objects,
        "drop_locations":drop_locations,
        "user_input" : user_input
    }
)
    
    

print(response)
print(type(response))


content='{"object" : "cube", "location" : "Not Valid", "status" : "Location not available, please select another object from [\'drop zone 1\', \'drop zone 2\', \'drop zone 3\', \'drop zone 4\']" }' additional_kwargs={} response_metadata={'model': 'qwen2.5-coder:7b', 'created_at': '2026-02-18T15:16:09.590802Z', 'done': True, 'done_reason': 'stop', 'total_duration': 16158310700, 'load_duration': 98940500, 'prompt_eval_count': 988, 'prompt_eval_duration': 8745440000, 'eval_count': 56, 'eval_duration': 7035079600, 'logprobs': None, 'model_name': 'qwen2.5-coder:7b', 'model_provider': 'ollama'} id='lc_run--019c7152-6dd7-7510-8747-8c0d961badcc-0' tool_calls=[] invalid_tool_calls=[] usage_metadata={'input_tokens': 988, 'output_tokens': 56, 'total_tokens': 1044}
<class 'langchain_core.messages.ai.AIMessage'>


In [19]:
from langchain_core.output_parsers import JsonOutputParser

from langchain_core.prompts import ChatPromptTemplate

system_prompt = """
You are a extractor who will extract only the object and destination zone from a user prompt, 

your main tasks are as follows:
1. Remove all specials characters, unwanted spaces and lower case the user prompt.
2. From the user prompt extract the object that need to be moved or placed, and check whether it is in {available_objects}, if not 
    "object" : "Not Valid", "location" : "extracted_location", "status" : "Object 'extracted object' not available, please select another object from {available_objects}".
3. From the user prompt extract the location where the object need to be moved or placed, and check whether it is in {drop_locations}, if not 
    "object" : "extracted_object", "location" : "Not Valid", "status" : "Location not available, please select another object from {drop_locations}.
4. If The user enter a object that is in present in this [{moved_objects}] then
    "object" : "Not Valid", "location" : "extracted_location", "status" : "Object 'extracted object' has already moved cannot move again, please select another object from {available_objects}"
5. If the both extracted object and location not in {available_objects} and {drop_locations} respectively, then
    "object" : "Not Valid", "location" : "Not Valid", "status" : "Both object and location is not valid, please select another object from {available_objects} and location from {drop_locations}"
6. The user may join the names togther without space give more space, or there will be under score between object name, handle all that perfectly

{format_instruction}

"""

chat_history = [
    ('system',system_prompt),
    
    ('human','move cube to drop_zone1'),
    ('ai','{{ "object":"cube", "location":"drop zone 1", "status":"Valid" }}'),
    
    ('human','move BRIEF_CASE to dropzone1'),
    ('ai','{{ "object":"brief case", "location":"drop zone 1", "status":"Valid"  }}'),
    
    ('human','move ball to drop zone1'),
    ('ai','{{ "object":"Not Valid", "location":"drop zone 1", "status":" "Object ball not available, please select another object from {available_objects}" }}'),
    
    ('human','move pen to drop zone_2'),
    ('ai','{{ "object":"pen", "location":"drop zone 2", "status":"Valid"  }}'),
    
    ('human','move oil to drop zone3'),
    ('ai','{{"object" : "Not Valid", "location" : "drop zone 3", "status" : "Object oil has already moved cannot move again, please select another object from {available_objects}" }}'),
    
    ('human','move First aidkit to dropzone-5'),
    ('ai','{{"object" : "first aid kit", "location" : "Not Valid", "status" = "Location not available, please select another object from {drop_locations}" }}'),
    
    ('human','move ball to dropzone-5'),
    ('ai','{{"object" : "Not Valid", "location" : "Not Valid", "status" : "Both object and location is not valid, please select another object from {available_objects} and location from {drop_locations}" }}'),
    
    ('human','{user_input}')
]

available_objects = ["cube", "brief case", "first aid kit", "paracetamol", "sugar"]
drop_locations = ['drop zone 1', 'drop zone 2', 'drop zone 3', 'drop zone 4']

moved_objects = ['oil','sugar']
parser = JsonOutputParser()

template = ChatPromptTemplate(messages=chat_history, partial_variables={"format_instruction":parser.get_format_instructions()})

user_input = "move cube to drop zone-6"


chain = template | model | parser

response = chain.invoke(
    {
        "moved_objects" : moved_objects,
        "available_objects" : available_objects,
        "drop_locations":drop_locations,
        "user_input" : user_input
    },
)
# print(response)
# print(type(response.content))
print(response)



{'object': 'cube', 'location': 'Not Valid', 'status': "Location not available, please select another object from ['drop zone 1', 'drop zone 2', 'drop zone 3', 'drop zone 4']"}


In [20]:
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import PydanticOutputParser, StrOutputParser
from langchain_core.runnables import RunnableBranch, RunnableLambda, RunnableParallel, RunnablePassthrough

class classificationModelSchema(BaseModel):
    prompt_type : Literal['general','task related'] = Field(description="Type of the query")

classification_parser = PydanticOutputParser(pydantic_object=classificationModelSchema)

classification_template = PromptTemplate(template = "You are classification agent that classifies the user prompt. Analyze the prompt and classifies the prompt as 'general' or 'task related' \n{user_query} \n{format_instructions}",
                                         input_variables=['user_query'],
                                         partial_variables={'format_instructions':classification_parser.get_format_instructions()}
                                         )
user_input = "move cube to target"


system_prompt = """
You are a extractor who will extract only the object and destination zone from a user prompt, 

your main tasks are as follows:
1. Remove all specials characters, unwanted spaces and lower case the user prompt.
2. From the user prompt extract the object that need to be moved or placed, and check whether it is in {available_objects}, if not 
    "object" = "Not Valid", "location" = "extracted_location", "status" = "Object 'extracted object' not available, please select another object from {available_objects}".
3. From the user prompt extract the location where the object need to be moved or placed, and check whether it is in {drop_locations}, if not 
    "object" = "extracted_object", "location" = "Not Valid", "status" = "Location not available, please select another object from {drop_locations}.
4. If The user enter a object that is in present in this [{moved_objects}] then
    "object" = "Not Valid", "location" = "extracted_location", "status" = "Object 'extracted object' has already moved cannot move again, please select another object from {available_objects}"
5. If the both extracted object and location not in {available_objects} and {drop_locations} respectively, then
    "object" = "Not Valid", "location" = "Not Valid", "status" = "Both object and location is not valid, please select another object from {available_objects} and location from {drop_locations}"
6. The user may join the names togther without space give more space, or there will be under score between object name, handle all that perfectly


"""

chat_history = [
    ('system',system_prompt),
    
    ('human','move cube to drop_zone1'),
    ('ai','{{ "object":"cube", "location":"drop zone 1", "status":"Valid" }}'),
    
    ('human','move BRIEF_CASE to dropzone1'),
    ('ai','{{ "object":"brief case", "location":"drop zone 1", "status":"Valid"  }}'),
    
    ('human','move ball to drop zone1'),
    ('ai','{{ "object":"Not Valid", "location":"drop zone 1", "status":" "Object ball not available, please select another object from {available_objects}" }}'),
    
    ('human','move pen to drop zone_2'),
    ('ai','{{ "object":"pen", "location":"drop zone 2", "status":"Valid"  }}'),
    
    ('human','move oil to drop zone3'),
    ('ai','{{"object" = "Not Valid", "location" = "drop zone 3", "status" = "Object oil has already moved cannot move again, please select another object from {available_objects}" }}'),
    
    ('human','move First aidkit to dropzone-5'),
    ('ai','{{"object" = "first aid kit", "location" = "Not Valid", "status" = "Location not available, please select another object from {drop_locations}" }}'),
    
    ('human','move ball to dropzone-5'),
    ('ai','{{"object" = "Not Valid", "location" = "Not Valid", "status" = "Both object and location is not valid, please select another object from {available_objects} and location from {drop_locations}" }}'),
    
    ('human','{user_input}')
]

available_objects = ["cube", "brief case", "first aid kit", "paracetamol", "sugar"]
drop_locations = ['drop zone 1', 'drop zone 2', 'drop zone 3', 'drop zone 4']

moved_objects = ['oil','sugar']

template = ChatPromptTemplate(messages=chat_history)


available_objects_schema = Literal["cube", "brief case", "first aid kit", "paracetamol", "sugar", "Not Valid"]
drop_locations_schema = Literal['drop zone 1', 'drop zone 2', 'drop zone 3', 'drop zone 4', 'Not Valid']

class ExtractorSchema(BaseModel):
    object : available_objects_schema = Field(description="Extracted object from the query")
    location : drop_locations_schema = Field(description="Extracted location from the query")
    status : str = Field(description="Overall status of the operation")
    

struct_model = model.with_structured_output(ExtractorSchema)


prompt_class_chain = classification_template | model | classification_parser

general_parser = StrOutputParser()
general_model_template = PromptTemplate(
    template="you are helpfull assistant: \n {input}",
    input_variables=['input']
)

input_extractor = RunnableLambda(lambda x:x['input']['user_query'])
template_extractor_task = RunnableLambda(lambda x : {
        "moved_objects" : moved_objects,
        "available_objects" : available_objects,
        "drop_locations":drop_locations,
        "user_input" : x['input']['user_query']
    })

parallel_chain = RunnableParallel(
    {
        "prompt_type" : prompt_class_chain,
        "input" : RunnablePassthrough()
    }
)

task_related_chain = template | struct_model 
general_chain = general_model_template | model | general_parser

branch_chain = RunnableBranch(
    (lambda x : x['prompt_type'].prompt_type == "task related", template_extractor_task | task_related_chain),
    (lambda x : x['prompt_type'].prompt_type == "general", input_extractor | general_chain ),
    RunnableLambda(lambda x : "Enter better prompt idiot" )
)

chain = parallel_chain | branch_chain

user_input = "move oil to drop zone 1"

result = chain.invoke({
    "user_query":user_input
})
print(result)



object='Not Valid' location='drop zone 1' status="Object oil has already moved cannot move again, please select another object from ['cube', 'brief case', 'first aid kit', 'paracetamol', 'sugar']"


In [21]:
chain.get_graph().print_ascii()

         +----------------------------------+      
         | Parallel<prompt_type,input>Input |      
         +----------------------------------+      
                  ***            ***               
                **                  ***            
              **                       **          
   +----------------+                    **        
   | PromptTemplate |                     *        
   +----------------+                     *        
            *                             *        
            *                             *        
            *                             *        
     +------------+                       *        
     | ChatOllama |                       *        
     +------------+                       *        
            *                             *        
            *                             *        
            *                             *        
+----------------------+          +-------------+  
| PydanticOu

## Tools

In [25]:
from langchain_community.tools import DuckDuckGoSearchRun
search_tool = DuckDuckGoSearchRun()
results = search_tool.invoke('isl news')
print(results)

1 day ago · ISL Musings: Chhetri, Kanteerava show up; Mohun Bagan look favourites as East Bengal, Jamshedpur shine in opening week. Sunil Chhetri celebrates after scoring ... 33 minutes ago · Chennaiyin FC aims for victory against Mumbai City in ISL opener, featuring coach Clifford Miranda's fresh leadership. 4 days ago · ISL Season Preview: Better late than never? Indian football resets for a truncated season. The truncated ISL 2025-26 season will start on Saturday. (X | ISL). 1 day ago · ... ISL, visit - https://bit.ly/ISL_Website Follow us on Instagram - https ... news & updates on ISL, visit - https://bit.ly/ISL_Web... Follow us on ... 20 Jan 2026 · The key amendments were AIFF removing commercial activities by a club, promotion and marketing and ticketing from ISL rules, which now include only logistical ...


In [34]:
search_tool.args_schema.model_json_schema()

{'description': 'Input for the DuckDuckGo search tool.',
 'properties': {'query': {'description': 'search query to look up',
   'title': 'Query',
   'type': 'string'}},
 'required': ['query'],
 'title': 'DDGInput',
 'type': 'object'}

In [27]:
from langchain_community.tools import ShellTool
shell_tool = ShellTool()
result = shell_tool.invoke('whoami')
print(result)

Executing command:
 whoami
laptop-pt8ev06h\faroo



f:\Profession\Coding\Study\LLM\llm_env\Lib\site-packages\langchain_community\tools\shell\tool.py:33: UserWarning: The shell tool has no safeguards by default. Use at your own risk.
  warnings.warn(


### Custom tool

In [28]:
from langchain_core.tools import tool

@tool
def multiply(a:int, b:int) -> int:
    """"Multiply two numbers"""
    return a*b




In [30]:
multiply.args

{'a': {'title': 'A', 'type': 'integer'},
 'b': {'title': 'B', 'type': 'integer'}}

In [31]:
multiply.name

'multiply'

In [32]:
multiply.description

'"Multiply two numbers'

In [33]:
multiply.args_schema.model_json_schema()

{'description': '"Multiply two numbers',
 'properties': {'a': {'title': 'A', 'type': 'integer'},
  'b': {'title': 'B', 'type': 'integer'}},
 'required': ['a', 'b'],
 'title': 'multiply',
 'type': 'object'}

In [ ]:
from langchain_core.tools import StructuredTool
class MultiplyInput(BaseModel):
    a : int = Field(required=True, description="The first number to add")
    b : int = Field(required=True, description="The second number to add")
    
def multiply_func(a : int, b:int):
    return a*b

multiply_tool =  StructuredTool(
    name='multiply',
    description='Multiply two numbers',
    args_schema=MultiplyInput,
    func=multiply_func
)

multiply_tool.invoke(
    {
        'a' : 3,
        'b' : 2
    }
)

C:\Users\faroo\AppData\Local\Temp\ipykernel_22084\482812250.py:3: PydanticDeprecatedSince20: Using extra keyword arguments on `Field` is deprecated and will be removed. Use `json_schema_extra` instead. (Extra keys: 'required'). Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.12/migration/
  a : int = Field(required=True, description="The first number to add")
C:\Users\faroo\AppData\Local\Temp\ipykernel_22084\482812250.py:4: PydanticDeprecatedSince20: Using extra keyword arguments on `Field` is deprecated and will be removed. Use `json_schema_extra` instead. (Extra keys: 'required'). Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.12/migration/
  b : int = Field(required=True, description="The second number to add")


6

In [36]:
multiply_tool.args_schema.model_json_schema()

{'properties': {'a': {'description': 'The first number to add',
   'required': True,
   'title': 'A',
   'type': 'integer'},
  'b': {'description': 'The second number to add',
   'required': True,
   'title': 'B',
   'type': 'integer'}},
 'required': ['a', 'b'],
 'title': 'MultiplyInput',
 'type': 'object'}

In [43]:
from langchain_core.tools import BaseTool
from typing import Type

class MultiplyInput(BaseModel):
    a : int = Field(description="The first number to add")
    b : int = Field(description="The second number to add")
    
    
class MutliplyTool(BaseTool):
    name : str = "multiply",
    description : str = "Mulitply two numbers"
    args_schema : Type[BaseModel] = MultiplyInput,
    
    def _run(self, a:int, b:int) -> int:
        return a*b
    

mult_tool = MutliplyTool()

mult_tool.invoke({
    'a' :4,
    'b':2
})

TypeError: issubclass() arg 1 must be a class

## Tool Calling
